In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
sqlcontext=SQLContext(sc)

In [4]:
poll=spark.createDataFrame([(1,"pollingtilte","2020","2021","pti","people")]).toDF("poll_id","title","startDate","endDate","organzier","participants")

In [5]:
poll.show()

+-------+------------+---------+-------+---------+------------+
|poll_id|       title|startDate|endDate|organzier|participants|
+-------+------------+---------+-------+---------+------------+
|      1|pollingtilte|     2020|   2021|      pti|      people|
+-------+------------+---------+-------+---------+------------+



In [6]:
question=spark.createDataFrame([(0,"questiontitle","subjective",1)]).toDF("question_id","title","type","poll")

In [7]:
question.show()

+-----------+-------------+----------+----+
|question_id|        title|      type|poll|
+-----------+-------------+----------+----+
|          0|questiontitle|subjective|   1|
+-----------+-------------+----------+----+



In [11]:
poll.createOrReplaceTempView("polls")
question.createOrReplaceTempView("questions")

In [14]:
spark.sql("""SELECT * FROM questions INNER JOIN polls on (polls.poll_id)=(questions.poll) ORDER BY startDate ; """).explain()

== Physical Plan ==
*(6) Sort [startDate#14 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(startDate#14 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#200]
   +- *(5) SortMergeJoin [poll#60L], [cast(poll_id#12 as bigint)], Inner
      :- *(2) Sort [poll#60L ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(poll#60L, 200), ENSURE_REQUIREMENTS, [id=#187]
      :     +- *(1) Project [_1#49L AS question_id#57L, _2#50 AS title#58, _3#51 AS type#59, _4#52L AS poll#60L]
      :        +- *(1) Filter isnotnull(_4#52L)
      :           +- *(1) Scan ExistingRDD[_1#49L,_2#50,_3#51,_4#52L]
      +- *(4) Sort [cast(poll_id#12 as bigint) ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(cast(poll_id#12 as bigint), 200), ENSURE_REQUIREMENTS, [id=#193]
            +- *(3) Project [_1#0 AS poll_id#12, _2#1 AS title#13, _3#2 AS startDate#14, _4#3 AS endDate#15, _5#4 AS organzier#16, _6#5 AS participants#17]
               +- *(3) Filter isnotnull(_1#0)
         

In [27]:
#SQL query
spark.sql("""SELECT poll_id,startDate,endDate,question_id,type FROM questions INNER JOIN polls ON (questions.poll)=(polls.poll_id)\
ORDER BY startDate ;""").explain()

== Physical Plan ==
*(6) Sort [startDate#14 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(startDate#14 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#670]
   +- *(5) Project [poll_id#12, startDate#14, endDate#15, question_id#57L, type#59]
      +- *(5) SortMergeJoin [poll#60L], [cast(poll_id#12 as bigint)], Inner
         :- *(2) Sort [poll#60L ASC NULLS FIRST], false, 0
         :  +- Exchange hashpartitioning(poll#60L, 200), ENSURE_REQUIREMENTS, [id=#656]
         :     +- *(1) Project [_1#49L AS question_id#57L, _3#51 AS type#59, _4#52L AS poll#60L]
         :        +- *(1) Filter isnotnull(_4#52L)
         :           +- *(1) Scan ExistingRDD[_1#49L,_2#50,_3#51,_4#52L]
         +- *(4) Sort [cast(poll_id#12 as bigint) ASC NULLS FIRST], false, 0
            +- Exchange hashpartitioning(cast(poll_id#12 as bigint), 200), ENSURE_REQUIREMENTS, [id=#662]
               +- *(3) Project [_1#0 AS poll_id#12, _3#2 AS startDate#14, _4#3 AS endDate#15]
                  +- *(3) Fi

In [ ]:
#